In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import time

In [2]:
df = pd.read_parquet('C:/Users/Ticy/Desktop/移動平均/dfTime_TLC-2022.parquet', engine='pyarrow')
df = df.drop('year', axis=1)
print(df.shape)

(5100018, 9)


In [3]:
df.head()

,Name,month,day,hour,weekday,PULocationID,is_holiday,次數,7DaysTimes
15229301,lyft,1,1,0,5,3,True,16,1760.0
15229302,lyft,1,1,0,5,4,True,49,3041.0
15229303,lyft,1,1,0,5,5,True,1,229.0
15229304,lyft,1,1,0,5,6,True,1,480.0
15229305,lyft,1,1,0,5,7,True,115,9874.0


In [4]:
valid = pd.read_parquet('C:/Users/Ticy/Desktop/移動平均/dfTime_TLC-202301.parquet', engine='pyarrow')
valid = valid.drop('year', axis=1)

In [5]:
train = df[~df['month'].isin([11, 12])]
test = df[df['month'].isin([11, 12])]
valid = valid[valid['month'] == 1]

In [6]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
train['Name'] = label.fit_transform(train['Name'])
test['Name'] = label.fit_transform(test['Name'])
valid['Name'] = label.fit_transform(valid['Name'])

C:\Users\Ticy\AppData\Local\Temp\ipykernel_20136\3534902206.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Name'] = label.fit_transform(train['Name'])
C:\Users\Ticy\AppData\Local\Temp\ipykernel_20136\3534902206.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Name'] = label.fit_transform(test['Name'])


In [7]:
train.head()

,Name,month,day,hour,weekday,PULocationID,is_holiday,次數,7DaysTimes
15229301,0,1,1,0,5,3,True,16,1760.0
15229302,0,1,1,0,5,4,True,49,3041.0
15229303,0,1,1,0,5,5,True,1,229.0
15229304,0,1,1,0,5,6,True,1,480.0
15229305,0,1,1,0,5,7,True,115,9874.0


In [8]:
test.head()

,Name,month,day,hour,weekday,PULocationID,is_holiday,次數,7DaysTimes
19476551,0,11,1,0,1,3,False,11,2589.0
19476552,0,11,1,0,1,4,False,33,4668.0
19476553,0,11,1,0,1,6,False,3,633.0
19476554,0,11,1,0,1,7,False,75,14534.0
19476555,0,11,1,0,1,9,False,2,960.0


In [9]:
valid.head()

,Name,month,day,hour,weekday,PULocationID,is_holiday,次數,7DaysTimes
434763,0,1,1,0,6,2,True,1,4.0
434764,0,1,1,0,6,3,True,12,2403.0
434765,0,1,1,0,6,4,True,49,3440.0
434766,0,1,1,0,6,5,True,5,324.0
434767,0,1,1,0,6,6,True,3,626.0


In [10]:
train = train.rename(columns={'次數': 'count'})
test = test.rename(columns={'次數': 'count'})
valid = valid.rename(columns={'次數': 'count'})

In [11]:
X_train = train.drop('count', axis=1).values
y_train = train['count'].values
X_test = test.drop('count', axis=1).values
y_test = test['count'].values
X_valid = valid.drop('count', axis=1).values
y_valid = valid['count'].values

## scikit-learn

In [20]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# scaler = StandardScaler()
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)
X_valid_scaled = scaler.fit_transform(X_valid)

## LightGBM

In [21]:
# start_time = time.time()


# from sklearn.model_selection import GridSearchCV
# kf = KFold(n_splits=5, shuffle=True, random_state=0)
# param_grid = {'n_estimators': range(100, 501, 100),
#               'learning_rate': np.logspace(0, -2, 3), 
#               'max_depth': range(3, 11)}

# lgbm = LGBMRegressor(n_jobs=-1, 
#                      random_state=0, 
#                      verbose=-1)

# model_cv = GridSearchCV(lgbm, param_grid, cv=kf, scoring='r2')
# model_cv.fit(X_train_scaled, y_train)
# print(model_cv.best_params_, model_cv.best_score_)


# end_time = time.time()
# execution_time = end_time - start_time
# print("執行時間:", execution_time, "秒")

In [22]:
start_time = time.time()


model = LGBMRegressor(n_estimators=10000, 
                      learning_rate=0.01, 
                      max_depth=8, 
                      num_leaves=31,
                      n_jobs=-1, 
                      random_state=0, 
                      verbose=-1)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}\nR2: {r2}')


end_time = time.time()
execution_time = end_time - start_time
print("執行時間:", execution_time, "秒")

MAE: 10.36144098809429
R2: 0.9184244859505616
執行時間: 353.62714409828186 秒


In [23]:
prediction = model.predict(X_valid_scaled)
mean_absolute_error(y_valid, prediction)
r2 = r2_score(y_valid, prediction)
print(f'MAE: {mae}\nR2: {r2}')

MAE: 10.36144098809429
R2: 0.9087360328647837


## NN

In [ ]:
# start_time = time.time()


# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Input
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import BatchNormalization
# from tensorflow.keras.layers import LeakyReLU

# n_cols = X_train.shape[1]

# model = Sequential([Dense(14, input_shape=(n_cols,)),
#                     BatchNormalization(),
#                     LeakyReLU(),
#                     Dense(14, input_shape=(n_cols,)),
#                     BatchNormalization(),
#                     LeakyReLU(),
#                     Dense(1)
# ])

# model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['accuracy'])
# model.summary()

# # 擬合X, y
# history = model.fit(X_train, y_train, validation_split=0.3, batch_size=100, epochs=10, verbose=2)


# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.legend(['training', 'validation'], loc='upper left')
# plt.show()


# result = model.evaluate(X_test, y_test)
# print(result)


# end_time = time.time()
# execution_time = end_time - start_time
# print("執行時間:", execution_time, "秒")